In [1]:
import os
import re
import glob
import pandas as pd

INPUT_DIR = "/Users/teemtat/Documents/MUICT/RESEARCH/exports/10th (LargeDT margin3 Dynamic Weight adjust)"
PATTERN   = "df??_summary.csv"
OUT_CSV   = "ALL_summary.csv"

def extract_num_from_name(path):
    """ดึงเลขลำดับจากชื่อไฟล์ เช่น df03_summary.csv -> 3 เพื่อไว้ sort ตามลำดับ"""
    fname = os.path.basename(path)
    m = re.search(r"df(\d+)_summary\.csv", fname, flags=re.IGNORECASE)
    return int(m.group(1)) if m else 0

def merge_csv(pattern, input_dir, out_csv):
    # หาไฟล์ทั้งหมดตาม pattern และเรียงตามเลขลำดับ df01, df02, ... df10
    files = sorted(
        glob.glob(os.path.join(input_dir, pattern)),
        key=extract_num_from_name
    )
    if not files:
        raise FileNotFoundError(f"ไม่พบไฟล์ตามแพทเทิร์น: {os.path.join(input_dir, pattern)}")

    frames = []
    for f in files:
        df = pd.read_csv(f)
        # เติมคอลัมน์แหล่งที่มา (เช่น df01_summary)
        df.insert(0, "source_file", os.path.splitext(os.path.basename(f))[0])
        frames.append(df)

    # รวมทุกไฟล์เข้าด้วยกัน (คอลัมน์ไม่ตรงกันก็จะรวมโดยอัตโนมัติ)
    merged = pd.concat(frames, ignore_index=True, sort=False)

    # พยายามแปลงคอลัมน์ที่เป็นตัวเลขให้เป็น numeric (หากทำได้)
    for c in merged.columns:
        merged[c] = pd.to_numeric(merged[c], errors="ignore")

    # จัดลำดับแถวตามหมายเลขภาพ (ถ้ามีคอลัมน์ 'image' เป็น df01, df02, ...)
    if "image" in merged.columns:
        def img_num(x):
            if isinstance(x, str):
                m = re.search(r"df(\d+)", x, flags=re.IGNORECASE)
                return int(m.group(1)) if m else 0
            return 0
        merged = merged.sort_values(by="image", key=lambda s: s.map(img_num)).reset_index(drop=True)

    # บันทึกผลลัพธ์
    merged.to_csv(out_csv, index=False)
    # try:
    #     merged.to_excel(out_xlsx, index=False)
    # except Exception:
    #     # ถ้าไม่มี openpyxl ก็ยังใช้ไฟล์ CSV ได้
    #     pass

    print(f"รวมไฟล์สำเร็จ -> {out_csv}")

if __name__ == "__main__":
    merge_csv(PATTERN, INPUT_DIR, OUT_CSV)


รวมไฟล์สำเร็จ -> ALL_summary.csv


/var/folders/p5/82yy1xsd1555hs3mqbcvmf2h0000gn/T/ipykernel_57112/3686458143.py:37: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  merged[c] = pd.to_numeric(merged[c], errors="ignore")
